Import the necessary libraries...

In [ ]:
from ortools.linear_solver import pywraplp


In the **third example**, we optimise a toy conference schedule. This is a **one-day conference** with **two rooms**.

| Time | Room 1 | Room 2 |
| --- | --- | --- |
| 10:00 - 11:00 |        |        |
| 10:00 - 11:00 |        |        |
| 13:00 - 14:00 |        |        | 
| 14:00 - 15:00 |        |        | 
| 15:30 - 16:30 |        |        | 

A total of **10 talks** have been proposed by **4 speakers**:

Brian | Terence | Neil | Berners 
--- | --- | --- | --- 
2 | 3 | 2 | 3

We wish to **assign** the 10 talks to the 10 slots, such that:
* No talk is repeated.
* No room hosts two talks at the same time.
* A speaker is not present in two rooms at the same time
* A speaker does not offer two talks in a row.

Here is the data:

In [ ]:
data = {}
data['speakers'] = {
    'Brian': 2,
    'Terence': 3,
    'Neil': 2,
    'Berners': 3
}
data['rooms'] = {
    'Room1': 5,
    'Room2': 5
}


We thus proceed through the steps to *create*, *specify* and *solve* a **mixed integer programming** problem using Google's OR-Tools.

### 1. Create the Solver
First, create a solver for MIP programming, as a class instance:

In [ ]:
solver = pywraplp.Solver.CreateSolver(solver_id='CBC')

WARNING: 'SCIP' is the default MIP solver for OR-tools. But it has an **academic only licence**. We therefore use CBC, which is an alternative open-source MIP solver that can be leveraged by or-tools.

### 2. Create the Variables
Next, create talk-slot assignment integer-valued variables $$x_{speaker, talk, room, slot} \in \{0,1\},$$
where the support definitions for the indeces are trivial.

In [ ]:
x = {}  # Speaker, Talk, Room, Slot -> 10 talks x 10 slots = 100 variables
for speaker in data['speakers']:
    for talk_num in range(data['speakers'][speaker]):
        for room in data['rooms']:
            for slot in range(data['rooms'][room]):
                x[speaker, talk_num, room, slot] = solver.IntVar(0, 1, '')
print(f"You created {len(x)} varialbes")

### 3. Add Constraints
Each talk can only be assigned to a single slot:

In [ ]:
for speaker in data['speakers']:
    for talk_num in range(data['speakers'][speaker]):
        solver.Add(
            sum(
                [x[speaker, talk_num, i, j]
                    for i in data['rooms']
                    for j in range(data['rooms'][i])]) <= 1)

Each slot can only host one talk:

In [ ]:
for room in data['rooms']:
    for slot in range(data['rooms'][room]):
        solver.Add(
            sum(
                [x[i, j, room, slot]
                    for i in data['speakers']
                    for j in range(data['speakers'][i])]) <= 1)

A speaker cannot be present in two rooms at the same time:

In [ ]:
for speaker in data['speakers']:
    for slot in range(data['rooms']['Room1']):
        solver.Add(
            sum(
                [x[speaker, i, j, slot]
                    for i in range(data['speakers'][speaker])
                    for j in data['rooms'].keys()]) <= 1)

A speaker needs a break between talks:

In [ ]:
for speaker in data['speakers']:
    for slot in range(data['rooms']['Room1']-1):
        solver.Add(
            sum([x[speaker, i, j, k]
                    for i in range(data['speakers'][speaker])
                    for j in data['rooms'].keys()
                    for k in range(slot, slot+2)]) <= 1)

### 4. Objective
Create an objective function that aims to fill the schedule:

In [ ]:
solver.Maximize(sum(list(x.values())))

### 5. Execute Job
Solve the problem by running the solver. A ```status``` informs us whether the optimisation routine reached an Optimal (```=0```) or Feasible (```=1```) solution.

In [ ]:
status = solver.Solve()  # 0 Means we reaached optimal solution
print(f"The optimisation routine returned status {status}")

Here is a summary of the schedule:

In [ ]:
for slot in range(data['rooms']['Room1']):
    print(f"Slot {slot}:")
    for room in data['rooms']:
        for speaker in data['speakers']:
            for talk_num in range(data['speakers'][speaker]):
                if x[speaker, talk_num, room, slot].solution_value() > 0.5:
                    print(f"{room}: {speaker} with talk {talk_num}")
